In [ ]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License.

In [1]:
import numpy as np
import pandas as pd 
from stock_energy.missingprocessor import Processor
import pickle


In [3]:
## Load GENE dataset 
# fname='/Users/venkitesh_work/Documents/work/Sapient_AI/Data/Ousai_encoder_data/Saved_data/GENE_dataset/npy_files_ML_input/gene_ml.npy'
# ip_data = np.load(fname)
# print(ip_data.shape)

# f_info='/Users/venkitesh_work/Documents/work/Sapient_AI/Data/Ousai_encoder_data/Saved_data/GENE_dataset/npy_files_ML_input/columns_gene.txt'
# channel_names = np.loadtxt(f_info,delimiter=',',dtype=str)
# channel_names = [i for i in channel_names if i] # Remove null strings

## Load GA dataset 
# fname = '/Users/venkitesh_work/Documents/work/Sapient_AI/Data/Ousai_encoder_data/Saved_data/GA_dataset/npy_files_ML_input/ga_ml.npy'
fname = '/Users/venkitesh_work/Documents/work/Sapient_AI/Data/Ousai_encoder_data/Saved_data/GA_dataset/npy_files_ML_input/normed_data/ga_normed_ml.npy'

ip_data = np.load(fname)
print(ip_data.shape)

f_info='/Users/venkitesh_work/Documents/work/Sapient_AI/Data/Ousai_encoder_data/Saved_data/GA_dataset/npy_files_ML_input/columns_ga.txt'
channel_names = np.loadtxt(f_info,delimiter=',',dtype=str)
channel_names = [i for i in channel_names if i] # Remove null strings

t_size       = 100
train_size   = 800
seq_len      = t_size
channel_list =[1,2,6,12,21]

temp_data = [i[channel_list,:t_size].T for i in ip_data[:train_size]]
print(len(temp_data),temp_data[0].shape,seq_len)


(1218, 23, 1200)
800 (100, 5) 100


In [4]:
# Build Dataframe 
df=pd.DataFrame([])
for ch in channel_list:
    # print(channel_names[ch],ip_data[:,ch,0].shape)
    df[channel_names[ch]]=ip_data[:,ch,0]
    
df.head()

,density,kappa,vloop,tribot,poh
0,-0.427921,0.272658,1.310819,0.698438,2.227408
1,0.349329,-0.449078,-0.506142,-0.737280,-0.551125
2,-1.152869,0.270440,-0.209020,-0.172159,-1.026438
3,0.496606,-0.466923,-0.493408,-1.003304,0.319370
4,-1.186968,0.440634,0.992907,-0.041743,3.318063


In [5]:

# data_path = "../TimeGAN/data"
loc = "stock"
# # seq_len = 24
# df = pd.read_csv('{}/{}_data.csv'.format(data_path,loc), sep = ",")
types = ["continuous" for i in range(len(df.columns))]


P = Processor(types)
# Flip the data to make chronological data
ori_data = P.fit_transform(df)
ori_data = ori_data[::-1]

# temp_data = [ori_data[i:i + seq_len] for i in range(0, len(ori_data) - seq_len)]    

print(seq_len)
from fastNLP import DataSet
dataset = DataSet({"seq_len": [seq_len] * len(temp_data), "dyn": temp_data, "sta":[0]*len(temp_data)})
dic = {
    "train_set": dataset,
    "dynamic_processor": P,
    "static_processor": Processor([])
}
print(P.dim, len(temp_data))

# raise SystemExit
from utils.general import make_sure_path_exists
make_sure_path_exists("./data")
with open("./data/{}.pkl".format(loc), "wb") as f:
    pickle.dump(dic, f)

density 1 None continuous
kappa 1 None continuous
vloop 1 None continuous
tribot 1 None continuous
poh 1 None continuous
100
5 800
